# Exibição de informações dos parlamentares

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3 as sql
import plotly.express as px

import seaborn as sns
import csv
import squarify 

import plotly.express as px
import plotly.offline as pyo

pyo.init_notebook_mode()

import os

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity="all"

## Carga de infos básicas. 

In [ ]:
#!ls ../dados/tse/ibg

### Carga de parlamentares

In [ ]:
#Dados de parlamentares
con = sql.connect("../dados/sql/base_completa.db")
df_parlamentares=pd.read_sql("select * from cadastro",con)
con.close()

In [ ]:
df_parlamentares.info()

## Carga de municipios e votação

In [ ]:
#Dados de municípios
con = sql.connect("../dados/sql/base_completa.db")
df_municipios_votacao=pd.read_sql("select * from parl_votacao",con)
df_municipios=pd.read_sql("select * from municipios",con)
con.close()

In [ ]:
#df_municipios.columns[18]
#df_municipios.head()
df_municipios_votacao.info()
df_municipios.info()

## Carga de informações geograficas dos municípios

In [ ]:
# informação geográfica dos municípios
lista_uf=list(df_municipios_votacao["SG_UF"].unique())
df_mapa_brasil=pd.DataFrame()

In [ ]:
for uf in lista_uf:
    diretorio="../dados/ibge/shapes/"+uf
    arquivos=os.listdir(diretorio)
    shapefile = [arq for arq in arquivos if ".shp" in arq][0]
    arq=diretorio+"/"+shapefile
    df_uf= gpd.read_file(arq)
    df_uf["SG_UF"]=uf
    df_mapa_brasil=pd.concat([df_mapa_brasil,df_uf])
    

In [ ]:
#df_mapa_brasil.info()

In [ ]:
df_mapa_brasil=df_mapa_brasil.astype({"CD_GEOCODM":"float64"})
df_mapa_brasil.rename(columns={"CD_GEOCODM":"CODIGO_IBGE"}, inplace=True)
#df_mapa_brasil.info()

In [ ]:
#df_mapa_brasil.head()
#df_mapa_brasil.to_csv("../dados/ibge/mapa_brasil.csv")

# Funções de exibição

In [ ]:
# Plot estado e municipios com indicador
def plot_estado(estado,indicador,titulo, minimo,maximo, cor):
    vmin=minimo
    vmax=maximo
    fig, ax = plt.subplots(figsize=(12,8))
    estado.plot(ax=ax, column=indicador, cmap=cor, edgecolor="black", linewidth=0.2)
    sm = plt.cm.ScalarMappable(cmap=cor, norm=plt.Normalize(vmin=vmax, vmax=vmin))
    sm._A = []
    cbar = fig.colorbar(sm)
    plt.title(titulo, fontsize=40, color="grey")
    ax.axis('off')
    plt.axis("equal")
    plt.show()
    return

In [ ]:
def plot_treemap(estado,indicador,titulo, minimo,maximo, cor):
    estado["NOME_IDH"]=estado["NM_MUNICIPIO"]+"\n IDH - "+estado["IDHM_2010"].round(6).astype(str)+"\n Votos - "+(estado["PERC_VOTOS"]*100).round(2).astype(str)+"%"
    vmin=maximo
    vmax=minimo
    cmap = plt.cm.RdBu
    norm = plt.Normalize(vmin=vmax, vmax=vmin)
    colors = [cmap(norm(value)) for value in estado[indicador]]
    fig, ax = plt.subplots(figsize=(16,12))
    sm = plt.cm.ScalarMappable(cmap=cor, norm=plt.Normalize(vmin=vmax, vmax=vmin))
    sm._A = []
    cbar = fig.colorbar(sm)
    plt.title(titulo, fontsize=40, color="grey")
    ax.axis('off')
    plt.axis("equal")
    squarify.plot(ax=ax, sizes=estado['VOTOS_TOTAL_MUN'][0:20], 
                  label=estado['NOME_IDH'][0:20], 
                  color=colors[0:20],
                  alpha=.8 )
    

In [ ]:
def plot_px_treemap(estado,indicador,titulo, minimo,maximo, cor):
    estado["NOME_IDH"]=estado["NM_MUNICIPIO"]+"\n IDH - "+estado["IDHM_2010"].round(6).astype(str)+"\n Votos - "+(estado["PERC_VOTOS"]*100).round(2).astype(str)+"%"
    vmin=maximo
    vmax=minimo
    estado=estado[estado["VOTOS_TOTAL_MUN"]>0]
    fig = px.treemap(estado, path=['SG_UF', 'NM_MUNICIPIO'], values='VOTOS_TOTAL_MUN',
                  color='IDHM_2010', hover_data=['PERC_VOTOS'],
                  color_continuous_scale='RdBu',
                  range_color=[0,1],
                  title=titulo)
    fig.show()
    return

In [ ]:
def exibe_cadastro(cpf):
    df_parlamentar=df_parlamentares[df_parlamentares["CPF"]==cpf]
    valor_bens=df_parlamentar["VL_BENS"]
    campos=["NM_URNA_CANDIDATO","NM_CANDIDATO","DT_NASCIMENTO","DS_COR_RACA",
            "DS_GRAU_INSTRUCAO","DS_OCUPACAO"]
    parlamentar=df_parlamentar[campos].to_dict(orient="records")[0]
    parlamentar["BENS"]=valor_bens.round(2)
    print(parlamentar)
    return


In [ ]:
def exibe_votacao_estado(cpf, cpf_votacao):
    uf=cpf_votacao["SG_UF"].iloc[0]
    nome=cpf["NM_URNA_CANDIDATO"].iloc[0]
    cpf_municipios=df_mapa_brasil[df_mapa_brasil["SG_UF"]==uf]
    cpf_municipios=pd.merge(cpf_municipios,cpf_votacao, how="left", on="CODIGO_IBGE")
    cpf_municipios=pd.merge(cpf_municipios,df_municipios, how="left", on="CODIGO_IBGE")
    cpf_municipios=cpf_municipios.sort_values('VOTOS_TOTAL_MUN', ascending=False)
    indicador=indicador="IDHM_2010"
    print(cpf_municipios.columns)
    titulo=uf+" - "+"IDHM Municípios"
    minimo=0
    maximo=1
    cor="RdBu"
    plot_estado(cpf_municipios, indicador, titulo, minimo ,maximo, cor)
    indicador="VOTOS_TOTAL_MUN"
    titulo=nome+" - "+"Votos por município"
    minimo=0
    maximo=cpf_municipios[indicador].max()
    cor="Blues"
    plot_estado(cpf_municipios, indicador, titulo, minimo ,maximo, cor)
    indicador=indicador="IDHM_2010"
    titulo=nome+" - "+"IDHM e Votos por município"
    minimo=0
    maximo=1
    cor="RdBu"
    #plot_treemap(cpf_municipios, indicador, titulo, minimo ,maximo, cor)
    plot_px_treemap(cpf_municipios, indicador, titulo, minimo ,maximo, cor)
    return

# Escolhe randomicamente um CPF e chama as funções

In [ ]:
cpf=df_parlamentares["CPF"].sample(n=1).iloc[0]
cpf
df_cpf_votacao=df_municipios_votacao[df_municipios_votacao["CPF"]==cpf]
df_cpf=df_parlamentares[df_parlamentares["CPF"]==cpf]
exibe_cadastro(cpf)
exibe_votacao_estado(df_cpf, df_cpf_votacao)

In [ ]:
nome="ALUISIO GUIMARAES MENDES FILHO"
#nome="FRANCISCO PLINIO VALERIO TOMAZ"
cpf=df_parlamentares[df_parlamentares["NM_CANDIDATO"]==nome]["CPF"].iloc[0]
cpf
df_cpf_votacao=df_municipios_votacao[df_municipios_votacao["CPF"]==cpf]
df_cpf=df_parlamentares[df_parlamentares["CPF"]==cpf]
exibe_cadastro(cpf)
exibe_votacao_estado(df_cpf, df_cpf_votacao)

In [ ]:
df_cpf_votacao.info()
df_cpf_votacao.head()
df_cpf_votacao. describe()

In [ ]:
df_cpf

In [ ]:
df_teste=pd.DataFrame({"Estado":["RJ","RJ","RJ"], "Mun":["Rio","Niteroi","Campos"],
                      "Pop":[20,10,0]})

In [ ]:
df_teste

In [ ]:
x=df_teste[df_teste["Pop"]>0]
x

In [ ]:
fig = px.treemap(df_teste, path=['Estado', 'Mun'], values='Pop',
                  color='Pop', hover_data=['Mun'],
                  color_continuous_scale='RdBu',
                      range_color=[0,20],
                     title="Teste")
fig.show()